# Working Version (Vimshottari Dasha Lords : working )

In [ ]:
import swisseph as swe
from geopy.geocoders import Nominatim
from timezonefinder import TimezoneFinder
import pytz
from datetime import datetime, timedelta

# ----- Input -----
DOB = "2025-07-03"
TOB = "16:06"
LOCATION = "Surat ,Gujarat"

# ----- Get Geolocation -----
geolocator = Nominatim(user_agent="vedic_astrology_app") # Use a unique user agent
location = geolocator.geocode(LOCATION)
if not location:
    raise ValueError(f"Could not find location for '{LOCATION}'")
lat, lon = location.latitude, location.longitude

# ----- Get Timezone and Local Time to UTC -----
tf = TimezoneFinder()
timezone_str = tf.timezone_at(lat=lat, lng=lon)
if not timezone_str:
    raise ValueError(f"Could not find timezone for Latitude: {lat}, Longitude: {lon}")
local_tz = pytz.timezone(timezone_str)

# Parse DOB and TOB
naive_dt = datetime.strptime(f"{DOB} {TOB}", "%Y-%m-%d %H:%M")
local_dt = local_tz.localize(naive_dt, is_dst=None) # is_dst=None handles DST ambiguity
utc_dt = local_dt.astimezone(pytz.utc)

# ----- Set Swiss Ephemeris -----
# IMPORTANT: Set your ephemeris path correctly.
# swe.set_ephe_path('YOUR_SWISSEPHE_EPHE_PATH_HERE')
# For demonstration, assuming default path or ephe files are accessible.
# If you get errors about ephemeris files, this path is likely incorrect.
# swe.set_ephe_path('ephe') # Example if 'ephe' folder is in current directory

# Set Ayanamsa for Vedic (Sidereal) calculations
# Lahiri Ayanamsa is commonly used in India
# swe.set_ayanamsa_arc(swe.AYANAMSA_LAHIRI)

# Calculate Julian Day for UTC time
jd = swe.julday(utc_dt.year, utc_dt.month, utc_dt.day,
                utc_dt.hour + utc_dt.minute / 60.0 + utc_dt.second / 3600.0)

# ----- Ascendant and Houses (These are typically Tropical in Western astrology houses,
#       but if you want them sidereal, you'd need a specific flag for houses or
#       convert them manually if your house system doesn't support sidereal directly)
# For standard Vedic interpretation, the sign of the Ascendant will be sidereal if swe.set_ayanamsa_arc is used.
cusps, ascmc = swe.houses(jd, lat, lon, b'A') # 'A' for Placidus, commonly used. For North/South Indian, house system varies.
ascendant_long = ascmc[0] 
house_cusps_long = cusps # These are longitudes of house cusps

# ----- Planet Positions (Sidereal) -----
planets = {
    "Sun": swe.SUN, "Moon": swe.MOON, "Mars": swe.MARS, "Mercury": swe.MERCURY,
    "Jupiter": swe.JUPITER, "Venus": swe.VENUS, "Saturn": swe.SATURN,
    "Rahu": swe.MEAN_NODE, "Ketu": swe.MEAN_NODE # Rahu and Ketu are always opposite, so use MEAN_NODE for both
}
planet_positions_sidereal = {} # Storing sidereal longitudes

# Flags for sidereal calculation: swe.FLG_SWIEPH is for standard calculations, swe.FLG_SIDEREAL for Ayanamsa
flags = swe.FLG_SWIEPH | swe.FLG_SIDEREAL | swe.FLG_NONUT # Try using FLG_NONUT as suggested by the error

# swe.FLG_NOABERR might also cause an error if FLG_NOCENT does, so remove it if FLG_NONUT doesn't work alone.
for name, pid in planets.items():
    # For Rahu/Ketu, MEAN_NODE gives Rahu's position, Ketu is always 180 degrees opposite
    if name == "Ketu":
        pos_rahu = swe.calc(jd, swe.MEAN_NODE, flags)[0]
        pos_ketu = (pos_rahu[0] + 180) % 360 # Ketu is 180 degrees from Rahu
        planet_positions_sidereal[name] = (pos_ketu, pos_rahu[1], pos_rahu[2]) # (lon, lat, speed)
    else:
        pos = swe.calc(jd, pid, flags)[0] # pos will be (longitude, latitude, speed)
        planet_positions_sidereal[name] = pos

# ----- Nakshatra and Pada -----
NAKSHATRA_SPAN_DEG = 13 + (20/60) # 13 degrees 20 minutes
PADA_SPAN_DEG = NAKSHATRA_SPAN_DEG / 4 # 3 degrees 20 minutes

nakshatra_names = [
    "Ashwini", "Bharani", "Krittika", "Rohini", "Mrigashira", "Ardra", "Punarvasu",
    "Pushya", "Ashlesha", "Magha", "Purva Phalguni", "Uttara Phalguni", "Hasta",
    "Chitra", "Swati", "Vishakha", "Anuradha", "Jyeshtha", "Mula", "Purva Ashadha",
    "Uttara Ashadha", "Shravana", "Dhanishtha", "Shatabhisha", "Purva Bhadrapada",
    "Uttara Bhadrapada", "Revati"
]

def get_nakshatra_info(moon_long_sidereal):
    # Ensure moon_long_sidereal is within 0-360 range
    moon_long_sidereal = moon_long_sidereal % 360
    print("Moon sidereal",moon_long_sidereal)

    nak_index = int(moon_long_sidereal / NAKSHATRA_SPAN_DEG) # 0-based index
    print(f"\n Nakshtra index :{nak_index}")

    
    # Calculate remainder within the current nakshatra for pada calculation
    nakshatra_remainder_deg = moon_long_sidereal % NAKSHATRA_SPAN_DEG
    pada_index = int(nakshatra_remainder_deg / PADA_SPAN_DEG) # 0-based index for pada
    
    nakshatra_num = nak_index + 1 # 1-based nakshatra number
    nakshatra_name = nakshatra_names[nak_index]
    pada_num = pada_index + 1 # 1-based pada number
    
    return nakshatra_num, nakshatra_name, pada_num

moon_long_for_nakshatra = planet_positions_sidereal["Moon"][0] 
print(f"moon longitude : {moon_long_for_nakshatra}")
nakshatra_index_1based, nakshatra_name, nakshatra_pada = get_nakshatra_info(moon_long_for_nakshatra)

# Vimshottari Dasha Lords & Years
dasha_order = ["Ketu", "Venus", "Sun", "Moon", "Mars", "Rahu", "Jupiter", "Saturn", "Mercury"]
dasha_years = {"Ketu": 7, "Venus": 20, "Sun": 6, "Moon": 10, "Mars": 7, "Rahu": 18, "Jupiter": 16, "Saturn": 19, "Mercury": 17}

def calculate_vimshottari_dasha(moon_long_sidereal, dob_str):
    # Ensure moon_long_sidereal is within 0-360 range
    moon_long_sidereal = moon_long_sidereal % 360

    # Determine current Nakshatra for Dasha lord
    nakshatra_deg_span = 360 / 27 # This is NAKSHATRA_SPAN_DEG
    nak_index = int(moon_long_sidereal // nakshatra_deg_span) # 0-based index of the nakshatra
    
    current_dasha_lord = dasha_order[nak_index % 9]
    
    # Calculate percentage of current Nakshatra *remaining* (for Dasha balance)
    # The balance of the Dasha is calculated from the portion of the Nakshatra *remaining* at birth.
    deg_into_nakshatra = moon_long_sidereal % nakshatra_deg_span
    deg_remaining_in_nakshatra = nakshatra_deg_span - deg_into_nakshatra
    
    percent_remaining_in_nakshatra = deg_remaining_in_nakshatra / nakshatra_deg_span
    
    total_years_of_current_dasha = dasha_years[current_dasha_lord]
    
    years_left_in_current_dasha = percent_remaining_in_nakshatra * total_years_of_current_dasha
    
    # Calculate the start date of the current Dasha period
    # If years_left is 5 and total is 10, then 5 years have passed in this dasha.
    years_passed_in_current_dasha = total_years_of_current_dasha - years_left_in_current_dasha
    
    dob_dt = datetime.strptime(dob_str, "%Y-%m-%d")
    
    # Start date of the *current* Mahadasha for the native
    start_date_current_mahadasha = dob_dt - timedelta(days=years_passed_in_current_dasha * 365.25)

    # Generate the full Dasha sequence
    sequence = []
    # Find the index of the current Dasha lord in the dasha_order list
    start_idx = dasha_order.index(current_dasha_lord)
    
    current_cycle_start_date = start_date_current_mahadasha
    
    for i in range(9): # Generate one full cycle of 9 dashas
        dasha_lord = dasha_order[(start_idx + i) % 9]
        years = dasha_years[dasha_lord]
        dasha_end_date = current_cycle_start_date + timedelta(days=years * 365.25)
        sequence.append((dasha_lord, current_cycle_start_date.date(), dasha_end_date.date()))
        current_cycle_start_date = dasha_end_date
        
    return current_dasha_lord, years_left_in_current_dasha, sequence

dasha_info_lord, left_year_info, vimshottari_dasha_sequence = calculate_vimshottari_dasha(moon_long_for_nakshatra, DOB)


def get_rashi_from_nakshatra_pada(nakshatra_num, pada_num):
    """
    Determines the Vedic Moon sign (Chandra Rashi) based on Nakshatra number and Pada.

    Args:
        nakshatra_num (int): The 1-based Nakshatra number (1 to 27).
        pada_num (int): The 1-based Pada number within the Nakshatra (1 to 4).

    Returns:
        str: The name of the Vedic Moon sign (Rashi).
             Returns "Invalid Input" if nakshatra_num or pada_num are out of range.
    """
    if not (1 <= nakshatra_num <= 27) or not (1 <= pada_num <= 4):
        return "Invalid Input: Nakshatra number must be 1-27, Pada 1-4."

    # List of Vedic Rashi names in order
    rashi_names = [
        "Aries (Mesha)", "Taurus (Vrishabha)", "Gemini (Mithuna)", "Cancer (Karka)",
        "Leo (Simha)", "Virgo (Kanya)", "Libra (Tula)", "Scorpio (Vrishchika)",
        "Sagittarius (Dhanu)", "Capricorn (Makara)", "Aquarius (Kumbha)", "Pisces (Meena)"
    ]

    # Calculate the total 0-indexed pada count from the beginning of the zodiac
    # (Nakshatra_num - 1) * 4 gets the total padas completed before the current nakshatra
    # + (pada_num - 1) gets the 0-indexed pada within the current nakshatra
    total_pada_index = (nakshatra_num - 1) * 4 + (pada_num - 1)

    # Each Rashi has 9 padas. Divide the total_pada_index by 9 to get the 0-indexed Rashi.
    rashi_index = total_pada_index // 9

    if 0 <= rashi_index < 12:
        return rashi_names[rashi_index]
    else:
        # This case should ideally not be reached with valid nakshatra/pada inputs
        return "Error in calculation"

# --- Example Usage with your existing script's output ---
# Call the function
moon_sign_rashi = get_rashi_from_nakshatra_pada(nakshatra_index_1based, nakshatra_pada)


# ----- Print Results -----
print("\n--- Vedic Astrology Chart ---")
print(f"Date of Birth: {DOB}, Time of Birth: {TOB} ({timezone_str})")
print(f"Location: {LOCATION} | Latitude: {lat:.2f}, Longitude: {lon:.2f}")
print(f"Julian Day: {jd}\n")

print("--- Ascendant and Houses ---")
print(f"Ascendant (Lagna): {ascendant_long:.2f}°")
for i, cusp in enumerate(house_cusps_long):
    print(f"House {i+1} Cusp: {cusp:.2f}°")

print("\n--- Planetary Positions (Sidereal) ---")
for name, pos_data in planet_positions_sidereal.items():
    print(f"{name}: {pos_data[0]:.2f}° (Lon) | Lat: {pos_data[1]:.2f}° | Speed: {pos_data[2]:.2f}°/day")

print(f"\n--- Moon's Nakshatra ---")
print(f"Nakshatra Name: {nakshatra_name}")
print(f"Nakshatra Number: {nakshatra_index_1based}, Pada: {nakshatra_pada}")

print(f"\n--- Moon's Rashi (Sign) ---")
print(f"Based on Nakshatra {nakshatra_name} (No. {nakshatra_index_1based}), Pada {nakshatra_pada}:")
print(f"Moon Sign (Chandra Rashi): {moon_sign_rashi}")

print(f"\n--- Current Vimshottari Mahadasha ---")
print(f"Current Dasha Lord: {dasha_info_lord}")
print(f"Years Left in {dasha_info_lord} Dasha: {left_year_info:.2f} years")

print("\n--- Full Vimshottari Mahadasha Sequence ---")
for dasha, start_date, end_date in vimshottari_dasha_sequence:
    print(f"{dasha}: {start_date} → {end_date}")

In [77]:
import swisseph as swe
from geopy.geocoders import Nominatim
from timezonefinder import TimezoneFinder
import pytz
from datetime import datetime, timedelta

# --- Input ---
DOB = "2004-07-14"
TOB = "07:15"
LOCATION = "Surat, Gujarat"

# --- Geolocation & Timezone ---
geo = Nominatim(user_agent="vedic_astro")
loc = geo.geocode(LOCATION)
lat, lon = loc.latitude, loc.longitude
tz = TimezoneFinder().timezone_at(lat=lat, lng=lon)
dt_naive = datetime.strptime(f"{DOB} {TOB}", "%Y-%m-%d %H:%M")
dt_local = pytz.timezone(tz).localize(dt_naive)
dt_utc = dt_local.astimezone(pytz.utc)

# --- Sidereal Julian Day & Ayanamsa ---
jd = swe.julday(dt_utc.year, dt_utc.month, dt_utc.day,
                dt_utc.hour + dt_utc.minute / 60)
swe.set_ayanamsa(swe.get_ayanamsa(jd))

# --- Panchang Constants ---
tithi_names = ["Pratipada","Dvitiya","Tritiya","Chaturthi","Panchami",
               "Shashthi","Saptami","Ashtami","Navami","Dashami",
               "Ekadashi","Dwadashi","Trayodashi","Chaturdashi","Purnima/Amavasya"]
karan_names = ["Kimstughna","Bava","Balava","Kaulava","Taitila","Garaja",
               "Vanija","Vishti","Shakuni","Chatushpada","Naga","Kimstughna"]
yoga_names = ["Vishkumbha","Priti","Ayushman","Saubhagya","Shobhana",
              "Atiganda","Sukarma","Dhriti","Shula","Ganda","Vriddhi",
              "Dhruva","Vyaghata","Harshana","Vajra","Siddhi","Vyatipata",
              "Variyana","Parigha","Shiva","Siddha","Sadhya","Shubha",
              "Shukla","Brahma","Indra","Vaidhriti"]

nak_list = ["Ashwini","Bharani","Krittika","Rohini","Mrigashira","Ardra",
            "Punarvasu","Pushya","Ashlesha","Magha","Purva Phalguni","Uttara Phalguni",
            "Hasta","Chitra","Swati","Vishakha","Anuradha","Jyeshtha","Mula",
            "Purva Ashadha","Uttara Ashadha","Shravana","Dhanishtha","Shatabhisha",
            "Purva Bhadrapada","Uttara Bhadrapada","Revati"]

# --- Panchang Functions ---
def get_panchang(jd, lat, lon):
    # Tithi, Nakshatra, Yoga, Karan
    tithi = int((swe.calc(jd, swe.MOON, swe.FLG_SIDEREAL)[0][0] -
                 swe.calc(jd, swe.SUN, swe.FLG_SIDEREAL)[0][0]) // 12) % 30
    nak = int(swe.calc(jd, swe.MOON, swe.FLG_SIDEREAL)[0][0] // (360/27))
    yoga = int((swe.calc(jd, swe.SUN, swe.FLG_SIDEREAL)[0][0] +
                swe.calc(jd, swe.MOON, swe.FLG_SIDEREAL)[0][0]) // (360/27))
    karan = tithi * 2
    return tithi_names[tithi], yoga_names[yoga % 27], karan_names[karan % 11], nak_list[nak]

# --- Avakhada Lookup Tables ---
varna = ["Brahmin","Kshatriya","Vaishya","Shudra"]
vashya = ["Man","Goddess","Beast","Deity","Human","Planet","Serpent","Mixed"]
yoni = ["Horse","Elephant","Rat","Sheep","Cow","Buffalo","Tiger","Deer","Dog","Cat","Monkey","Lion","Female"]
gan = ["Deva","Manushya","Rakshasa"]
nadi = ["Adi","Madhya","Antya"]
tatva = ["Akash","Vayu","Tejas","Ap","Prithvi"]
paya = ["Gaja","Vrishabha","Simha","Vrisha","Meena","Karma","Shobhana","Goma","Gaja","Vrishabha","Simha","Vrisha"]

def get_avakhada(nak_idx, pada):
    v_idx = nak_idx // 7
    return {
        "varna": varna[v_idx],
        "vashya": vashya[nak_idx % len(vashya)],
        "yoni": yoni[nak_idx % len(yoni)],
        "gan": gan[nak_idx % 3],
        "nadi": nadi[pada % 3],
        "tatva": tatva[pada % 5],
        "paya": paya[pada % len(paya)]
    }

# --- Nakshatra + Pada + Avakhada ---
moon_lon = swe.calc(jd, swe.MOON, swe.FLG_SIDEREAL)[0][0] % 360
nak_idx = int(moon_lon // (360/27))
pada = int((moon_lon % (360/27)) // ((360/27)/4))
nak_name, tithi, yoga, karan = nak_list[nak_idx], *get_panchang(jd, lat, lon)[0:3], nak_list[nak_idx]
avakh = get_avakhada(nak_idx, pada)

# --- Vimshottari Dasha (Indian-style, accurate 360-day) ---
lords = ["Ketu","Venus","Sun","Moon","Mars","Rahu","Jupiter","Saturn","Mercury"]
years = {"Ketu":7,"Venus":20,"Sun":6,"Moon":10,"Mars":7,"Rahu":18,"Jupiter":16,"Saturn":19,"Mercury":17}
dasha_idx = nak_idx % 9
rem_frac = 1 - ((moon_lon % (360/27)) / (360/27))
dob_dt = datetime.strptime(DOB, "%Y-%m-%d")

seq = []
start = dob_dt
for i in range(9):
    lord = lords[(dasha_idx + i) % 9]
    dur = int(rem_frac * years[lord] * 360) if i == 0 else years[lord] * 360
    end = start + timedelta(days=dur)
    seq.append((lord, start.date(), end.date()))
    start = end
    rem_frac = 1  # only first is partial

# --- Output ---
print(f"Panchang: Tithi={tithi}, Yog={yoga}, Karan={karan}, Nakshatra={nak_name} Pada={pada+1}")
print("Avakhada:", avakh)
print("\nVimshottari Dasha:")
for lord, s, e in seq:
    print(f"{lord:<8}: {s} → {e}")


AttributeError: module 'swisseph' has no attribute 'set_ayanamsa'

# 

In [2]:
import swisseph as swe
from geopy.geocoders import Nominatim
from timezonefinder import TimezoneFinder
import pytz
from datetime import datetime, timedelta

# --- Configuration ---
try:
    swe.set_ephe_path('ephe') # Try relative path first
except Exception as e:
    print(f"Error setting ephemeris path to 'ephe': {e}")
    print("Please set swe.set_ephe_path('FULL_PATH_TO_YOUR_SWEPH_EPHE_FOLDER') manually.")

# --- Global Data / Constants ---
RASHI_NAMES = [
    "Aries (Mesha)", "Taurus (Vrishabha)", "Gemini (Mithuna)", "Cancer (Karka)",
    "Leo (Simha)", "Virgo (Kanya)", "Libra (Tula)", "Scorpio (Vrishchika)",
    "Sagittarius (Dhanu)", "Capricorn (Makara)", "Aquarius (Kumbha)", "Pisces (Meena)"
]

NAKSHATRA_NAMES = [
    "Ashwini", "Bharani", "Krittika", "Rohini", "Mrigashira", "Ardra", "Punarvasu",
    "Pushya", "Ashlesha", "Magha", "Purva Phalguni", "Uttara Phalguni", "Hasta",
    "Chitra", "Swati", "Vishakha", "Anuradha", "Jyeshtha", "Mula", "Purva Ashadha",
    "Uttara Ashadha", "Shravana", "Dhanishtha", "Shatabhisha", "Purva Bhadrapada",
    "Uttara Bhadrapada", "Revati"
]

TITHI_NAMES = [
    "Pratipada", "Dwitiya", "Tritiya", "Chaturthi", "Panchami", "Shashti", "Saptami",
    "Ashtami", "Navami", "Dashami", "Ekadashi", "Dwadashi", "Trayodashi", "Chaturdashi",
    "Purnima", # Full Moon (Shukla Paksha 15th)
    "Pratipada", "Dwitiya", "Tritiya", "Chaturthi", "Panchami", "Shashti", "Saptami",
    "Ashtami", "Navami", "Dashami", "Ekadashi", "Dwadashi", "Trayodashi", "Chaturdashi",
    "Amavasya" # New Moon (Krishna Paksha 15th)
]
PAKSHA_NAMES = ["Shukla Paksha (Waxing)", "Krishna Paksha (Waning)"]

CHARA_KARANS = ["Bava", "Balava", "Kaulava", "Taitila", "Garija", "Vanija", "Vishti"]
STHIRA_KARANS = ["Shakuni", "Chatushpada", "Naga", "Kimstughna"]

YOGA_NAMES = [
    "Vishkambha", "Priti", "Ayushman", "Saubhagya", "Shobhana", "Atiganda", "Sukarma",
    "Dhriti", "Shula", "Ganda", "Vriddhi", "Dhruva", "Vyaghata", "Harshana", "Vajra",
    "Siddhi", "Vyatipata", "Variyana", "Parigha", "Shiva", "Siddha", "Sadhya", "Shubha",
    "Shukla", "Brahma", "Indra", "Vaidhriti"
]

DASH_ORDER = ["Ketu", "Venus", "Sun", "Moon", "Mars", "Rahu", "Jupiter", "Saturn", "Mercury"]
DASH_YEARS = {"Ketu": 7, "Venus": 20, "Sun": 6, "Moon": 10, "Mars": 7, "Rahu": 18, "Jupiter": 16, "Saturn": 19, "Mercury": 17}

NAKSHATRA_SPAN_DEG = 13 + (20/60) # 13 degrees 20 minutes
PADA_SPAN_DEG = NAKSHATRA_SPAN_DEG / 4 # 3 degrees 20 minutes

# --- Avakhada Lookup Tables ---

# Varna (based on Nakshatra Index 0-26) - CORRECTED
VARNA_MAPPING_NAKSHATRA = [
    "Vaishya", "Shudra", "Vaishya", "Shudra", "Mleccha", "Mleccha", "Brahmin", "Shudra", "Mleccha", # Ashwini to Ashlesha
    "Shudra", "Shudra", "Vaishya", "Vaishya", "Brahmin", "Mleccha", "Mleccha", "Brahmin", "Shudra", # Magha to Jyeshtha
    "Mleccha", "Mleccha", "Vaishya", "Shudra", "Mleccha", "Brahmin", "Mleccha", "Mleccha", "Vaishya" # Mula to Revati
    # Note: "Mleccha" is sometimes used for a mixed category or outside the traditional four.
    # More common mappings group some differently, but this is one standard interpretation.
    # For a strictly 4-varna mapping (Brahmin, Kshatriya, Vaishya, Shudra):
    # Ashwini: Vaishya, Bharani: Shudra, Krittika: Vaishya, Rohini: Shudra, Mrigashira: Shudra, Ardra: Shudra,
    # Punarvasu: Brahmin, Pushya: Shudra, Ashlesha: Shudra, Magha: Shudra, P. Phalguni: Kshatriya, U. Phalguni: Kshatriya,
    # Hasta: Vaishya, Chitra: Shudra, Swati: Vaishya, Vishakha: Kshatriya, Anuradha: Shudra, Jyeshtha: Shudra,
    # Mula: Shudra, P. Ashadha: Shudra, U. Ashadha: Kshatriya, Shravana: Shudra, Dhanishtha: Shudra, Shatabhisha: Shudra,
    # P. Bhadrapada: Shudra, U. Bhadrapada: Shudra, Revati: Vaishya.
    # I'll use a more common 4-varna mapping to be precise for the user's request.

    # Revised VARNA_MAPPING_NAKSHATRA for traditional 4 Varnas:
    # Source: Many traditional texts, e.g., Brihat Parashara Hora Shastra interpretations.
    "Vaishya",   # Ashwini
    "Shudra",    # Bharani
    "Vaishya",   # Krittika
    "Shudra",    # Rohini
    "Shudra",    # Mrigashira
    "Shudra",    # Ardra
    "Brahmin",   # Punarvasu
    "Shudra",    # Pushya
    "Shudra",    # Ashlesha
    "Shudra",    # Magha
    "Kshatriya", # Purva Phalguni
    "Kshatriya", # Uttara Phalguni
    "Vaishya",   # Hasta
    "Shudra",    # Chitra
    "Vaishya",   # Swati
    "Kshatriya", # Vishakha
    "Shudra",    # Anuradha
    "Shudra",    # Jyeshtha
    "Shudra",    # Mula
    "Shudra",    # Purva Ashadha
    "Kshatriya", # Uttara Ashadha
    "Shudra",    # Shravana
    "Shudra",    # Dhanishtha
    "Shudra",    # Shatabhisha
    "Shudra",    # Purva Bhadrapada
    "Shudra",    # Uttara Bhadrapada
    "Vaishya"    # Revati
]


# Vashya (based on Moon Rashi) - This mapping is generally correct.
VASHYA_MAPPING = {
    "Aries (Mesha)": "Chatushpada (Quadruped)",
    "Taurus (Vrishabha)": "Chatushpada (Quadruped)",
    "Gemini (Mithuna)": "Dwipada (Human)",
    "Cancer (Karka)": "Jalchar (Aquatic)",
    "Leo (Simha)": "Vanachar (Forest Dweller)",
    "Virgo (Kanya)": "Dwipada (Human)",
    "Libra (Tula)": "Dwipada (Human)",
    "Scorpio (Vrishchika)": "Keet (Insect)",
    "Sagittarius (Dhanu)": "Chatushpada (Quadruped)", # Can be Dwipada for last half, simplified here
    "Capricorn (Makara)": "Jalchar (Aquatic)", # Can be Chatushpada for last half, simplified here
    "Aquarius (Kumbha)": "Dwipada (Human)",
    "Pisces (Meena)": "Jalchar (Aquatic)"
}

# Yoni (based on Nakshatra Index 0-26) - This mapping is generally correct for the 14 animals.
# The list cycles through the 14 Yonis.
YONI_MAPPING = [
    "Ashwa (Horse)",    # Ashwini
    "Gaja (Elephant)",  # Bharani
    "Mesha (Sheep)",    # Krittika
    "Sarpa (Serpent)",  # Rohini
    "Mriga (Deer)",     # Mrigashira
    "Shwaan (Dog)",     # Ardra
    "Marjara (Cat)",    # Punarvasu
    "Mushaka (Mouse)",  # Pushya
    "Simha (Lion)",     # Ashlesha
    "Mahisha (Buffalo)",# Magha
    "Vyaghra (Tiger)",  # Purva Phalguni
    "Vanara (Monkey)",  # Uttara Phalguni
    "Nakula (Mongoose)",# Hasta
    "Vyaghra (Tiger)",  # Chitra (Repeats)
    "Mahisha (Buffalo)",# Swati (Repeats)
    "Mriga (Deer)",     # Vishakha (Repeats)
    "Shwaan (Dog)",     # Anuradha (Repeats)
    "Gaja (Elephant)",  # Jyeshtha (Repeats)
    "Ashwa (Horse)",    # Mula (Repeats)
    "Marjara (Cat)",    # Purva Ashadha (Repeats)
    "Nakula (Mongoose)",# Uttara Ashadha (Repeats)
    "Vanara (Monkey)",  # Shravana (Repeats)
    "Simha (Lion)",     # Dhanishtha (Repeats)
    "Sarpa (Serpent)",  # Shatabhisha (Repeats)
    "Mesha (Sheep)",    # Purva Bhadrapada (Repeats)
    "Mushaka (Mouse)",  # Uttara Bhadrapada (Repeats)
    "Gaja (Elephant)"   # Revati (Repeats - assuming same as Jyeshtha for some classifications)
]

# Gan (based on Nakshatra Index 0-26) - Correct.
GAN_NAKSHATRA_MAP = [
    "Deva", "Manushya", "Deva", "Manushya", "Rakshasa", "Manushya", "Deva", "Deva", "Manushya", "Manushya",
    "Manushya", "Rakshasa", "Deva", "Deva", "Rakshasa", "Rakshasa", "Manushya", "Deva", "Manushya", "Rakshasa",
    "Manushya", "Deva", "Manushya", "Rakshasa", "Manushya", "Rakshasa", "Deva"
]

# Nadi (based on Nakshatra Index 0-26) - Correct.
NADI_NAKSHATRA_MAP = [
    "Adya (Vata)", "Madhya (Pitta)", "Antya (Kapha)", "Antya (Kapha)", "Madhya (Pitta)", "Adya (Vata)",
    "Antya (Kapha)", "Madhya (Pitta)", "Adya (Vata)", "Antya (Kapha)", "Madhya (Pitta)", "Adya (Vata)",
    "Antya (Kapha)", "Madhya (Pitta)", "Adya (Vata)", "Antya (Kapha)", "Madhya (Pitta)", "Adya (Vata)",
    "Antya (Kapha)", "Madhya (Pitta)", "Adya (Vata)", "Antya (Kapha)", "Madhya (Pitta)", "Adya (Vata)",
    "Antya (Kapha)", "Madhya (Pitta)", "Adya (Vata)"
]

# Sign Lord (based on Rashi Name) - Correct.
SIGN_LORD_MAPPING = {
    "Aries (Mesha)": "Mars (Mangal)",
    "Taurus (Vrishabha)": "Venus (Shukra)",
    "Gemini (Mithuna)": "Mercury (Buddha)",
    "Cancer (Karka)": "Moon (Chandra)",
    "Leo (Simha)": "Sun (Surya)",
    "Virgo (Kanya)": "Mercury (Buddha)",
    "Libra (Tula)": "Venus (Shukra)",
    "Scorpio (Vrishchika)": "Mars (Mangal)",
    "Sagittarius (Dhanu)": "Jupiter (Guru)",
    "Capricorn (Makara)": "Saturn (Shani)",
    "Aquarius (Kumbha)": "Saturn (Shani)",
    "Pisces (Meena)": "Jupiter (Guru)"
}

# Tatva (Elements based on Rashi) - Correct.
TATVA_MAPPING = {
    "Aries (Mesha)": "Agni (Fire)", "Leo (Simha)": "Agni (Fire)", "Sagittarius (Dhanu)": "Agni (Fire)",
    "Taurus (Vrishabha)": "Prithvi (Earth)", "Virgo (Kanya)": "Prithvi (Earth)", "Capricorn (Makara)": "Prithvi (Earth)",
    "Gemini (Mithuna)": "Vayu (Air)", "Libra (Tula)": "Vayu (Air)", "Aquarius (Kumbha)": "Vayu (Air)",
    "Cancer (Karka)": "Jala (Water)", "Scorpio (Vrishchika)": "Jala (Water)", "Pisces (Meena)": "Jala (Water)"
}

# Paya (based on Nakshatra group) - Correct.
PAYA_NAKSHATRA_MAP = [
    "Swarna (Gold)", "Rajata (Silver)", "Tamra (Copper)", "Loha (Iron)", "Rajata (Silver)", "Tamra (Copper)",
    "Rajata (Silver)", "Tamra (Copper)", "Swarna (Gold)", "Rajata (Silver)", "Tamra (Copper)", "Loha (Iron)",
    "Rajata (Silver)", "Tamra (Copper)", "Swarna (Gold)", "Loha (Iron)", "Tamra (Copper)", "Rajata (Silver)",
    "Loha (Iron)", "Tamra (Copper)", "Rajata (Silver)", "Tamra (Copper)", "Loha (Iron)", "Swarna (Gold)",
    "Swarna (Gold)", "Rajata (Silver)", "Loha (Iron)"
]


# --- Helper Functions ---

def format_longitude_to_dms(longitude_deg):
    """Converts total degrees (0-360) to Zodiac Sign, Degree, Minute, Second format."""
    total_degrees = longitude_deg % 360 

    sign_index_0based = int(total_degrees / 30)
    degrees_in_sign_decimal = total_degrees % 30

    degrees = int(degrees_in_sign_decimal)
    minutes_decimal = (degrees_in_sign_decimal - degrees) * 60
    minutes = int(minutes_decimal)
    seconds = int((minutes_decimal - minutes) * 60)

    sign_name = RASHI_NAMES[sign_index_0based]
    
    return f"{degrees}° {minutes}' {seconds}\" {sign_name.split(' ')[0]}"

def get_nakshatra_info(moon_long_sidereal):
    """Determines Nakshatra number, name, and Pada from Moon's sidereal longitude."""
    moon_long_sidereal = moon_long_sidereal % 360
    
    nak_index = int(moon_long_sidereal / NAKSHATRA_SPAN_DEG) 
    
    nakshatra_remainder_deg = moon_long_sidereal % NAKSHATRA_SPAN_DEG
    pada_index = int(nakshatra_remainder_deg / PADA_SPAN_DEG) 
    
    nakshatra_num = nak_index + 1 
    nakshatra_name = NAKSHATRA_NAMES[nak_index]
    pada_num = pada_index + 1 
    
    return nak_index, nakshatra_num, nakshatra_name, pada_num 

def get_rashi_from_nakshatra_pada(nakshatra_num, pada_num):
    """Determines the Vedic Moon sign (Chandra Rashi) based on Nakshatra number and Pada."""
    if not (1 <= nakshatra_num <= 27) or not (1 <= pada_num <= 4):
        return "Invalid Input: Nakshatra number must be 1-27, Pada 1-4."

    total_pada_index = (nakshatra_num - 1) * 4 + (pada_num - 1)
    rashi_index = total_pada_index // 9

    if 0 <= rashi_index < 12:
        return RASHI_NAMES[rashi_index]
    else:
        return "Error in calculation"

def calculate_vimshottari_dasha(moon_long_sidereal, dob_datetime_obj): 
    """
    Calculates the Vimshottari Mahadasha sequence.
    Args:
        moon_long_sidereal (float): Moon's sidereal longitude at birth.
        dob_datetime_obj (datetime.datetime): UTC datetime object of birth.
    Returns:
        tuple: (current_dasha_lord, years_left_in_current_dasha, full_dasha_sequence)
    """
    moon_long_sidereal = moon_long_sidereal % 360

    nakshatra_deg_span = 360 / 27 
    nak_index = int(moon_long_sidereal // nakshatra_deg_span) 
    
    current_dasha_lord = DASH_ORDER[nak_index % 9]
    
    deg_into_nakshatra = moon_long_sidereal % nakshatra_deg_span
    deg_remaining_in_nakshatra = nakshatra_deg_span - deg_into_nakshatra
    
    percent_remaining_in_nakshatra = deg_remaining_in_nakshatra / nakshatra_deg_span
    
    total_years_of_current_dasha = DASH_YEARS[current_dasha_lord]
    
    years_left_in_current_dasha = percent_remaining_in_nakshatra * total_years_of_current_dasha
    
    years_passed_in_current_dasha = total_years_of_current_dasha - years_left_in_current_dasha
    
    start_date_current_mahadasha = dob_datetime_obj - timedelta(days=years_passed_in_current_dasha * 365.25)

    sequence = []
    start_idx = DASH_ORDER.index(current_dasha_lord)
    
    current_cycle_start_date = start_date_current_mahadasha
    
    for i in range(9): # Generate one full cycle of 9 dashas
        dasha_lord = DASH_ORDER[(start_idx + i) % 9]
        years = DASH_YEARS[dasha_lord]
        dasha_end_date = current_cycle_start_date + timedelta(days=years * 365.25)
        sequence.append((dasha_lord, current_cycle_start_date.date(), dasha_end_date.date()))
        current_cycle_start_date = dasha_end_date
        
    return current_dasha_lord, years_left_in_current_dasha, sequence

def get_karan(moon_long_sidereal, sun_long_sidereal):
    """
    Calculates the Karan based on the current Tithi number and whether it's the first or second half.
    There are 60 karans in a lunar month. Kimstughna appears at the start of Shukla Pratipada
    and again at the end of Krishna Chaturdashi/Amavasya.
    The 7 movable karans (Bava, Balava, Kaulava, Taitila, Garija, Vanija, Vishti) repeat 8 times.
    The 4 fixed karans (Shakuni, Chatushpada, Naga, Kimstughna) appear once each at specific points.
    """
    phase_angle = (moon_long_sidereal - sun_long_sidereal + 360) % 360
    
    karan_index_in_cycle = int(phase_angle / 6.0) 
    
    if karan_index_in_cycle == 0:
        return STHIRA_KARANS[3] # Kimstughna (initial)
    elif 1 <= karan_index_in_cycle <= 56: # 7 movable karans repeat 8 times
        return CHARA_KARANS[(karan_index_in_cycle - 1) % 7]
    elif karan_index_in_cycle == 57:
        return STHIRA_KARANS[0] # Shakuni
    elif karan_index_in_cycle == 58:
        return STHIRA_KARANS[1] # Chatushpada
    elif karan_index_in_cycle == 59:
        return STHIRA_KARANS[2] # Naga 
    else:
        return "N/A" 


def get_yoga(sun_long_sidereal, moon_long_sidereal):
    """Calculates the Yoga based on sum of Sun's and Moon's longitudes."""
    yoga_longitude_total = (sun_long_sidereal + moon_long_sidereal) % 360
    
    yoga_index = int(yoga_longitude_total / NAKSHATRA_SPAN_DEG) 
    
    if 0 <= yoga_index < 27:
        return YOGA_NAMES[yoga_index]
    else:
        return "N/A" 

def get_vara_tithi_karan_yoga(jd_ut, sun_long_sidereal, moon_long_sidereal):
    """Calculates Vara (Weekday), Tithi, Karan, and Yoga."""
    
    # Vara (Weekday)
    revjul_result = swe.revjul(jd_ut)
    
    year, month, day, hour = 0, 0, 0, 0 
    minute, second = 0, 0 

    if len(revjul_result) == 7: 
        year, month, day, hour, minute, second, swe_weekday_num = revjul_result
    elif len(revjul_result) == 4: 
        year, month, day, hour = revjul_result
        swe_weekday_num = 0 
    else:
        print(f"Warning: swe.revjul returned {len(revjul_result)} values. Expected 4 or 7. Cannot reliably determine weekday.")
        swe_weekday_num = 0 
        
    try:
        dt_utc_for_vara = datetime(int(year), int(month), int(day), int(hour), int(minute), int(second), tzinfo=pytz.utc)
        python_weekday = dt_utc_for_vara.weekday() 
        vara_index = (python_weekday + 1) % 7 
    except ValueError as e:
        print(f"Error converting Julian Day to datetime for weekday calculation: {e}. Defaulting to Sunday.")
        vara_index = 0 

    vara_names = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
    vara = vara_names[vara_index]
    
    # Tithi 
    phase_angle = (moon_long_sidereal - sun_long_sidereal + 360) % 360
    tithi_index_raw = int(phase_angle / 12) 
    
    paksha = ""
    tithi_name = ""
    
    if tithi_index_raw < 15: 
        paksha = PAKSHA_NAMES[0]
        tithi_name = TITHI_NAMES[tithi_index_raw]
    else:
        paksha = PAKSHA_NAMES[1]
        tithi_name = TITHI_NAMES[tithi_index_raw % 15] 
        
    # Karan
    karan = get_karan(moon_long_sidereal, sun_long_sidereal)

    # Yoga
    yoga = get_yoga(sun_long_sidereal, moon_long_sidereal)

    return vara, tithi_name, paksha, karan, yoga

def get_avakhada_details(nakshatra_idx, moon_rashi_name, nakshatra_pada, yoga_name, karan_name, tithi_name_str):
    """
    Calculates Avakhada details based on provided astrological data.
    Args:
        nakshatra_idx (int): 0-based index of the Moon Nakshatra.
        moon_rashi_name (str): The name of the Moon Sign (Chandra Rashi).
        nakshatra_pada (int): The 1-based Nakshatra Pada.
        yoga_name (str): The calculated Yoga name.
        karan_name (str): The calculated Karan name.
        tithi_name_str (str): The calculated Tithi name string (e.g., "Pratipada (Shukla Paksha)").
    Returns:
        dict: A dictionary containing all Avakhada details.
    """
    avakhada = {}

    # Varna (based on Nakshatra Index) - CORRECTED
    if 0 <= nakshatra_idx < len(VARNA_MAPPING_NAKSHATRA):
        avakhada["Varna"] = VARNA_MAPPING_NAKSHATRA[nakshatra_idx]
    else:
        avakhada["Varna"] = "N/A"

    avakhada["Vashya"] = VASHYA_MAPPING.get(moon_rashi_name, "N/A")

    # Yoni (based on Nakshatra Index)
    if 0 <= nakshatra_idx < len(YONI_MAPPING):
        avakhada["Yoni"] = YONI_MAPPING[nakshatra_idx]
    else:
        avakhada["Yoni"] = "N/A"

    # Gan (based on Nakshatra Index)
    if 0 <= nakshatra_idx < len(GAN_NAKSHATRA_MAP):
        avakhada["Gan"] = GAN_NAKSHATRA_MAP[nakshatra_idx]
    else:
        avakhada["Gan"] = "N/A"

    # Nadi (based on Nakshatra Index)
    if 0 <= nakshatra_idx < len(NADI_NAKSHATRA_MAP):
        avakhada["Nadi"] = NADI_NAKSHATRA_MAP[nakshatra_idx]
    else:
        avakhada["Nadi"] = "N/A"

    # Sign (Moon Sign)
    avakhada["Sign"] = moon_rashi_name

    # Sign Lord (of Moon Sign)
    avakhada["Sign Lord"] = SIGN_LORD_MAPPING.get(moon_rashi_name, "N/A")

    # Nakshatra Charan (Pada)
    avakhada["Nakshatra Charan (pada)"] = nakshatra_pada

    # Yoga, Karan, Tithi (Already calculated, just assign)
    avakhada["Yoga"] = yoga_name
    avakhada["Karan"] = karan_name
    avakhada["Tithi"] = tithi_name_str 

    # Yunja - REMOVED as it's typically synonymous with Yoni or not a distinct Avakhada element.
    # avakhada["Yunja"] = yoga_name # Removed

    # Tatva (Element based on Moon Rashi)
    avakhada["Tatva"] = TATVA_MAPPING.get(moon_rashi_name, "N/A")

    # Paya (based on Nakshatra Index)
    if 0 <= nakshatra_idx < len(PAYA_NAKSHATRA_MAP):
        avakhada["Paya"] = PAYA_NAKSHATRA_MAP[nakshatra_idx]
    else:
        avakhada["Paya"] = "N/A"

    return avakhada


# --- Main Calculation Logic ---

# ----- Input -----
DOB = "2004-07-14"
TOB = "07:15"
LOCATION = "Surat ,Gujarat"

# ----- Get Geolocation -----
geolocator = Nominatim(user_agent="vedic_astrology_app") 
location = geolocator.geocode(LOCATION)
if not location:
    raise ValueError(f"Could not find location for '{LOCATION}'")
lat, lon = location.latitude, location.longitude

# ----- Get Timezone and Local Time to UTC -----
tf = TimezoneFinder()
timezone_str = tf.timezone_at(lat=lat, lng=lon)
if not timezone_str:
    raise ValueError(f"Could not find timezone for Latitude: {lat}, Longitude: {lon}")
local_tz = pytz.timezone(timezone_str)

naive_dt = datetime.strptime(f"{DOB} {TOB}", "%Y-%m-%d %H:%M")
local_dt = local_tz.localize(naive_dt, is_dst=None)
utc_dt = local_dt.astimezone(pytz.utc)

# Calculate Julian Day for UTC time
jd = swe.julday(utc_dt.year, utc_dt.month, utc_dt.day,
                utc_dt.hour + utc_dt.minute / 60.0 + utc_dt.second / 3600.0)

# --- Ayanamsa Setup ---
ayanamsa_flag_for_calc = 0 
ayanamsa_used_display = "Unknown / Default" 

try:
    if hasattr(swe, 'AYANAMSA_LAHIRI') and callable(getattr(swe, 'set_ayanamsa_arc', None)):
        swe.set_ayanamsa_arc(swe.AYANAMSA_LAHIRI)
        ayanamsa_used_display = "Lahiri (via swe.set_ayanamsa_arc)"
    else:
        print("\nWARNING: Modern Ayanamsa setting functions (swe.set_ayanamsa_arc, swe.AYANAMSA_LAHIRI) not found.")
        print("This indicates your pyswisseph version is outdated.")
        print("Attempting to use legacy Ayanamsa flag directly in swe.calc (assuming Lahiri = 2).")
        ayanamsa_flag_for_calc = 2 
        ayanamsa_used_display = "Lahiri (via legacy direct flag)"
except AttributeError as e:
    print(f"\nCRITICAL WARNING: Encountered AttributeError during Ayanamsa setup: {e}")
    print("Your pyswisseph library is severely outdated.")
    print("Please run: pip install --upgrade pyswisseph")
    print("If error persists after upgrade, verify your Python environment.")
    ayanamsa_flag_for_calc = 2 
    ayanamsa_used_display = "Lahiri (via forced legacy flag due to error)"


# ----- Ascendant and Houses -----
cusps, ascmc = swe.houses(jd, lat, lon, b'A') 
ascendant_long = ascmc[0] 
house_cusps_long = cusps 

# ----- Planet Positions (Sidereal) -----
planets_ids = {
    "Sun": swe.SUN, "Moon": swe.MOON, "Mars": swe.MARS, "Mercury": swe.MERCURY,
    "Jupiter": swe.JUPITER, "Venus": swe.VENUS, "Saturn": swe.SATURN,
    "Rahu": swe.MEAN_NODE, "Ketu": swe.MEAN_NODE
}
planet_positions_sidereal = {} 

flags = swe.FLG_SWIEPH | swe.FLG_SIDEREAL | ayanamsa_flag_for_calc | swe.FLG_NONUT 

for name, pid in planets_ids.items():
    if name == "Ketu":
        pos_rahu = swe.calc(jd, swe.MEAN_NODE, flags)[0]
        pos_ketu_lon = (pos_rahu[0] + 180) % 360 
        planet_positions_sidereal[name] = (pos_ketu_lon, pos_rahu[1], pos_rahu[2]) 
    else:
        pos = swe.calc(jd, pid, flags)[0] 
        planet_positions_sidereal[name] = pos

# ----- Panchang Calculations -----
sun_long_sidereal = planet_positions_sidereal["Sun"][0]
moon_long_sidereal = planet_positions_sidereal["Moon"][0]

vara, tithi_name_simple, paksha_name, karan_name, yoga_name = get_vara_tithi_karan_yoga(jd, sun_long_sidereal, moon_long_sidereal)
full_tithi_name = f"{tithi_name_simple} ({paksha_name})"

# ----- Nakshatra, Pada, and Rashi for Moon -----
nakshatra_idx_0based, nakshatra_index_1based, nakshatra_name, nakshatra_pada = get_nakshatra_info(moon_long_sidereal)
moon_sign_rashi = get_rashi_from_nakshatra_pada(nakshatra_index_1based, nakshatra_pada)

# ----- Vimshottari Dasha -----
dasha_info_lord, left_year_info, vimshottari_dasha_sequence = calculate_vimshottari_dasha(moon_long_sidereal, utc_dt)

# ----- Avakhada Details -----
avakhada_details = get_avakhada_details(
    nakshatra_idx_0based,
    moon_sign_rashi,
    nakshatra_pada,
    yoga_name,
    karan_name,
    full_tithi_name
)

# --- Print Results ---
print("\n" + "="*70)
print("             Vedic Astrology Birth Chart Report             ")
print("="*70 + "\n")

print("--- Birth Details ---")
print(f"Date of Birth: {DOB}")
print(f"Time of Birth: {TOB} ({timezone_str} / {utc_dt.strftime('%H:%M UTC')})")
print(f"Location: {LOCATION}")
print(f"Coordinates: Latitude: {lat:.2f}°, Longitude: {lon:.2f}°")
print(f"Julian Day: {jd:.4f}\n")
print(f"Ayanamsa Used: {ayanamsa_used_display}\n")


print("--- Panchang Details ---")
print(f"{'Vara (Weekday)':<20}: {vara}")
print(f"{'Tithi':<20}: {full_tithi_name}")
print(f"{'Nakshatra':<20}: {nakshatra_name} (No. {nakshatra_index_1based})")
print(f"Nakshatra Pada: {nakshatra_pada}")
print(f"{'Yoga':<20}: {yoga_name}")
print(f"{'Karan':<20}: {karan_name}\n")


print("--- Ascendant and Houses ---")
ascendant_rashi, asc_deg, asc_min, asc_sec = format_longitude_to_dms(ascendant_long).split(' ') 
print(f"Ascendant (Lagna): {asc_deg} {asc_min} {asc_sec} {ascendant_rashi}")
for i, cusp in enumerate(house_cusps_long):
    cusp_rashi, cusp_deg, cusp_min, cusp_sec = format_longitude_to_dms(cusp).split(' ')
    print(f"House {i+1} Cusp: {cusp_deg} {cusp_min} {cusp_sec} {cusp_rashi}")

print("\n--- Planetary Positions (Sidereal) ---")
for name, pos_data in planet_positions_sidereal.items():
    lon_str = format_longitude_to_dms(pos_data[0])
    print(f"{name:<10}: {lon_str:<20} | Lat: {pos_data[1]:.2f}° | Speed: {pos_data[2]:.2f}°/day")

print(f"\n--- Moon's Details ---")
print(f"Moon's Nakshatra: {nakshatra_name} (No. {nakshatra_index_1based})")
print(f"Moon Sign (Chandra Rashi): {moon_sign_rashi}\n")


print("--- Avakhada Details ---")
for key, value in avakhada_details.items():
    print(f"{key:<20}: {value}")

print(f"\n--- Vimshottari Mahadasha ---")
print(f"Current Dasha Lord: {dasha_info_lord}")
print(f"Years Left in {dasha_info_lord} Dasha: {left_year_info:.2f} years")

print("\n--- Full Vimshottari Mahadasha Sequence (One Cycle) ---")
for dasha, start_date, end_date in vimshottari_dasha_sequence:
    print(f"{dasha:<7}: {start_date} → {end_date}")

print("\n" + "="*70)
print("              End of Birth Chart Report             ")
print("="*70)


This indicates your pyswisseph version is outdated.
Attempting to use legacy Ayanamsa flag directly in swe.calc (assuming Lahiri = 2).

             Vedic Astrology Birth Chart Report             

--- Birth Details ---
Date of Birth: 2004-07-14
Time of Birth: 07:15 (Asia/Kolkata / 01:45 UTC)
Location: Surat ,Gujarat
Coordinates: Latitude: 21.21°, Longitude: 72.83°
Julian Day: 2453200.5729

Ayanamsa Used: Lahiri (via legacy direct flag)

--- Panchang Details ---
Vara (Weekday)      : Wednesday
Tithi               : Dwadashi (Krishna Paksha (Waning))
Nakshatra           : Rohini (No. 4)
Nakshatra Pada: 3
Yoga                : Vriddhi
Karan               : Taitila

--- Ascendant and Houses ---
Ascendant (Lagna): 29' 27" Leo 6°
House 1 Cusp: 29' 27" Leo 6°
House 2 Cusp: 29' 27" Virgo 6°
House 3 Cusp: 29' 27" Libra 6°
House 4 Cusp: 29' 27" Scorpio 6°
House 5 Cusp: 29' 27" Sagittarius 6°
House 6 Cusp: 29' 27" Capricorn 6°
House 7 Cusp: 29' 27" Aquarius 6°
House 8 Cusp: 29' 27" Pisces 6°
Ho

In [6]:
import swisseph as swe
from geopy.geocoders import Nominatim
from timezonefinder import TimezoneFinder
import pytz
from datetime import datetime, timedelta
import math


# 🌗 Convert decimal to DMS
def decimal_to_dms(deg):
    d = int(deg)
    m = int((deg - d) * 60)
    s = int(((deg - d) * 60 - m) * 60)
    return f"{d}° {m}' {s}\""

# ♈ Sign names and lords
signs = [
    "Aries", "Taurus", "Gemini", "Cancer", "Leo", "Virgo",
    "Libra", "Scorpio", "Sagittarius", "Capricorn", "Aquarius", "Pisces"
]

sign_lords = {
    "Aries": "Mars", "Taurus": "Venus", "Gemini": "Mercury", "Cancer": "Moon",
    "Leo": "Sun", "Virgo": "Mercury", "Libra": "Venus", "Scorpio": "Mars",
    "Sagittarius": "Jupiter", "Capricorn": "Saturn", "Aquarius": "Saturn", "Pisces": "Jupiter"
}

# 🌟 Nakshatras and their lords
nakshatra_list = [
    "Ashwini", "Bharani", "Krittika", "Rohini", "Mrigashira", "Ardra", "Punarvasu",
    "Pushya", "Ashlesha", "Magha", "Purva Phalguni", "Uttara Phalguni", "Hasta",
    "Chitra", "Swati", "Vishakha", "Anuradha", "Jyeshtha", "Mula", "Purva Ashadha",
    "Uttara Ashadha", "Shravana", "Dhanishta", "Shatabhisha", "Purva Bhadrapada",
    "Uttara Bhadrapada", "Revati"
]

nakshatra_lords = [
    "Ketu", "Venus", "Sun", "Moon", "Mars", "Rahu", "Jupiter",
    "Saturn", "Mercury", "Ketu", "Venus", "Sun", "Moon",
    "Mars", "Rahu", "Jupiter", "Saturn", "Mercury", "Ketu",
    "Venus", "Sun", "Moon", "Mars", "Rahu", "Jupiter", "Saturn", "Mercury"
]

# 👶 Avastha logic
def get_avastha(deg_in_sign, sign_number):
    avasthas = ["Bala (Child)", "Kumara (Teen)", "Yuva (Adult)", "Vriddha (Old)", "Mrita (Dead)"]
    index = int(deg_in_sign // 6)
    return avasthas[index] if sign_number % 2 == 1 else avasthas[::-1][index]

# 🔥 Combustion logic
def is_combust(planet_long, sun_long):
    diff = abs((planet_long - sun_long + 180) % 360 - 180)  # Normalize
    return diff < 8  # You can refine per planet

# 📜 Determine sign & nakshatra
def get_sign(longitude):
    index = int(longitude / 30)
    return signs[index], index + 1

def get_nakshatra(longitude):
    nak_index = int(longitude / 13.3333)
    return nakshatra_list[nak_index], nakshatra_lords[nak_index]

# 🪐 Main processor function
def get_all_planet_details(jd, lat, lon):
    swe.set_sid_mode(swe.SIDM_LAHIRI)
    flags = swe.FLG_SWIEPH | swe.FLG_SIDEREAL | swe.FLG_NONUT

    cusps, ascmc = swe.houses(jd, lat, lon, b'P')
    ascendant_long = ascmc[0]

    # Add ASC as a pseudo-planet
    planet_ids = {
        "Ascendant": ascendant_long,
        "Sun": swe.SUN, "Moon": swe.MOON, "Mars": swe.MARS, "Mercury": swe.MERCURY,
        "Jupiter": swe.JUPITER, "Venus": swe.VENUS, "Saturn": swe.SATURN,
        "Rahu": swe.MEAN_NODE, "Ketu": swe.MEAN_NODE, "Uranus": swe.URANUS,
        "Neptune": swe.NEPTUNE, "Pluto": swe.PLUTO
    }

    # Get sun longitude for combustion comparison
    sun_long = swe.calc(jd, swe.SUN, flags)[0][0]

    all_planets_data = {}

    for name, value in planet_ids.items():
        if name == "Ascendant":
            longitude = value
        elif name == "Ketu":
            rahu_pos = swe.calc(jd, swe.MEAN_NODE, flags)[0][0]
            longitude = (rahu_pos + 180) % 360
        else:
            longitude = swe.calc(jd, value, flags)[0][0]

        sign, sign_num = get_sign(longitude)
        deg_in_sign = longitude % 30
        dms = decimal_to_dms(deg_in_sign)

        nakshatra, nak_lord = get_nakshatra(longitude)
        avastha = get_avastha(deg_in_sign, sign_num)
        combust = "Yes" if name != "Sun" and is_combust(longitude, sun_long) else "No"
        sign_lord = sign_lords[sign]

        # Find house number
        house_num = None
        for i in range(12):
            start = cusps[i]
            end = cusps[(i + 1) % 12]
            if start < end:
                if start <= longitude < end:
                    house_num = i + 1
                    break
            else:  # wrap
                if longitude >= start or longitude < end:
                    house_num = i + 1
                    break

        all_planets_data[name] = {
            "house_number": house_num,
            "Longitude": round(longitude, 2),
            "Degree in sign": round(deg_in_sign, 2),
            "DMS": dms,
            "Sign": sign,
            "SignLord": sign_lord,
            "Nakshatra": nakshatra,
            "NakLord": nak_lord,
            "Avastha": avastha,
            "Combust": combust,
            "Status": ""  # Optionally calculate based on planet friendships
        }

    return all_planets_data

# User Inputs
DOB = "2004-07-14"
TOB = "07:15"
LOCATION = "Surat, Gujarat"

# Geolocation and Time
geolocator = Nominatim(user_agent="vedic_astrology")
location = geolocator.geocode(LOCATION)
lat, lon = location.latitude, location.longitude

# Timezone and UTC conversion
tf = TimezoneFinder()
timezone_str = tf.timezone_at(lat=lat, lng=lon)
local_tz = pytz.timezone(timezone_str)
naive_dt = datetime.strptime(f"{DOB} {TOB}", "%Y-%m-%d %H:%M")
local_dt = local_tz.localize(naive_dt)
utc_dt = local_dt.astimezone(pytz.utc)

# Swiss Ephemeris setup
swe.set_sid_mode(swe.SIDM_LAHIRI)
jd = swe.julday(utc_dt.year, utc_dt.month, utc_dt.day,
                utc_dt.hour + utc_dt.minute / 60.0 + utc_dt.second / 3600.0)
# --- Calculation Flags ---
flags = (
    swe.FLG_SWIEPH     # Swiss ephemeris computation
    | swe.FLG_SIDEREAL # Sidereal zodiac (for Vedic)
    | swe.FLG_NONUT    # Ignore nutation (used for pure sidereal)
    | swe.FLG_SPEED    # Return velocity of planets too (for combustion/retro checks)
)

cusps, ascmc = swe.houses(jd, lat, lon, b'P')
ascendant = ascmc[0]

planet_det = get_all_planet_details(jd,lat,lon)
from pprint import pprint 
pprint(planet_det)



{'Ascendant': {'Avastha': 'Kumara (Teen)',
               'Combust': 'No',
               'DMS': '6° 29\' 27"',
               'Degree in sign': 6.49,
               'Longitude': 126.49,
               'NakLord': 'Ketu',
               'Nakshatra': 'Magha',
               'Sign': 'Leo',
               'SignLord': 'Sun',
               'Status': '',
               'house_number': 1},
 'Jupiter': {'Avastha': 'Vriddha (Old)',
             'Combust': 'No',
             'DMS': '21° 27\' 46"',
             'Degree in sign': 21.46,
             'Longitude': 141.46,
             'NakLord': 'Venus',
             'Nakshatra': 'Purva Phalguni',
             'Sign': 'Leo',
             'SignLord': 'Sun',
             'Status': '',
             'house_number': 1},
 'Ketu': {'Avastha': 'Yuva (Adult)',
          'Combust': 'No',
          'DMS': '13° 27\' 19"',
          'Degree in sign': 13.46,
          'Longitude': 193.46,
          'NakLord': 'Rahu',
          'Nakshatra': 'Swati',
          'Sig

In [ ]:
ascendant = 126.49089058401412

# Step 1: Sign list
signs = [
    "Aries", "Taurus", "Gemini", "Cancer", "Leo", "Virgo",
    "Libra", "Scorpio", "Sagittarius", "Capricorn", "Aquarius", "Pisces"
]

# Step 2: Calculate values
sign_index = int(ascendant // 30)
sign_name = signs[sign_index]
degree_in_sign = ascendant % 30

# Step 3: DMS conversion
def decimal_to_dms(deg_float):
    degrees = int(deg_float)
    minutes_float = (deg_float - degrees) * 60
    minutes = int(minutes_float)
    seconds = int((minutes_float - minutes) * 60)
    return f"{degrees}° {minutes}' {seconds}\""

dms = decimal_to_dms(degree_in_sign)

# Step 4: Print Result
print("Ascendant Details:")
print(f"🔸 Longitude: {ascendant:.2f}°")
print(f"🔸 Sign: {sign_name}")
print(f"🔸 Degree in Sign: {degree_in_sign:.6f}")
print(f"🔸 DMS in Sign: {dms}")


: 

# Navamsa calculation

In [ ]:
def get_navamsa_sign(sign, degree_in_sign):
    navamsa_sign_sequence = {
        "Aries": ["Aries", "Taurus", "Gemini", "Cancer", "Leo", "Virgo", "Libra", "Scorpio", "Sagittarius"],
        "Taurus": ["Capricorn", "Aquarius", "Pisces", "Aries", "Taurus", "Gemini", "Cancer", "Leo", "Virgo"],
        "Gemini": ["Libra", "Scorpio", "Sagittarius", "Capricorn", "Aquarius", "Pisces", "Aries", "Taurus", "Gemini"],
        "Cancer": ["Aries", "Taurus", "Gemini", "Cancer", "Leo", "Virgo", "Libra", "Scorpio", "Sagittarius"],
        "Leo": ["Sagittarius", "Capricorn", "Aquarius", "Pisces", "Aries", "Taurus", "Gemini", "Cancer", "Leo"],
        "Virgo": ["Capricorn", "Aquarius", "Pisces", "Aries", "Taurus", "Gemini", "Cancer", "Leo", "Virgo"],
        "Libra": ["Gemini", "Cancer", "Leo", "Virgo", "Libra", "Scorpio", "Sagittarius", "Capricorn", "Aquarius"],
        "Scorpio": ["Cancer", "Leo", "Virgo", "Libra", "Scorpio", "Sagittarius", "Capricorn", "Aquarius", "Pisces"],
        "Sagittarius": ["Aries", "Taurus", "Gemini", "Cancer", "Leo", "Virgo", "Libra", "Scorpio", "Sagittarius"],
        "Capricorn": ["Capricorn", "Aquarius", "Pisces", "Aries", "Taurus", "Gemini", "Cancer", "Leo", "Virgo"],
        "Aquarius": ["Libra", "Scorpio", "Sagittarius", "Capricorn", "Aquarius", "Pisces", "Aries", "Taurus", "Gemini"],
        "Pisces": ["Cancer", "Leo", "Virgo", "Libra", "Scorpio", "Sagittarius", "Capricorn", "Aquarius", "Pisces"]
    }

    navamsa_no = int(degree_in_sign // (30 / 9))
    navamsa_sign = navamsa_sign_sequence[sign][navamsa_no]
    return navamsa_sign, navamsa_no + 1

# Apply to your data:
planets = {
      "Jupiter": {
        "Avastha": "Vriddha (Old)",
        "Combust": "No",
        "DMS": "21\u00b0 27' 46\"",
        "Degree in sign": 21.46290003973047,
        "Longitude": "141.46\u00b0",
        "NakLord": "Venus",
        "Nakshatra": "Purva Phalguni",
        "Sign": "Leo",
        "SignLord": "Sun",
        "Status": "Friendly",
        "house": 5
      },
      "Ketu": {
        "Avastha": "Yuva (Adult)",
        "Combust": "No",
        "DMS": "13\u00b0 27' 19\"",
        "Degree in sign": 13.455328052325683,
        "Longitude": "193.46\u00b0",
        "NakLord": "Rahu",
        "Nakshatra": "Swati",
        "Sign": "Libra",
        "SignLord": "Venus",
        "Status": "Friendly",
        "house": 7
      },
      "Mars": {
        "Avastha": "Kumara (Teen)",
        "Combust": "No",
        "DMS": "18\u00b0 48' 26\"",
        "Degree in sign": 18.807258892004953,
        "Longitude": "108.81\u00b0",
        "NakLord": "Mercury",
        "Nakshatra": "Ashlesha",
        "Sign": "Cancer",
        "SignLord": "Moon",
        "Status": "Friendly",
        "house": 4
      },
      "Mercury": {
        "Avastha": "Kumara (Teen)",
        "Combust": "No",
        "DMS": "21\u00b0 34' 38\"",
        "Degree in sign": 21.57739052499629,
        "Longitude": "111.58\u00b0",
        "NakLord": "Mercury",
        "Nakshatra": "Ashlesha",
        "Sign": "Cancer",
        "SignLord": "Moon",
        "Status": "Enemy",
        "house": 4
      },
      "Moon": {
        "Avastha": "Kumara (Teen)",
        "Combust": "No",
        "DMS": "20\u00b0 50' 52\"",
        "Degree in sign": 20.84794852374023,
        "Longitude": "50.85\u00b0",
        "NakLord": "Moon",
        "Nakshatra": "Rohini",
        "Sign": "Taurus",
        "SignLord": "Venus",
        "Status": "Neutral",
        "house": 2
      },
      "Neptune": {
        "Avastha": "Kumara (Teen)",
        "Combust": "No",
        "DMS": "20\u00b0 40' 57\"",
        "Degree in sign": 20.682755036344588,
        "Longitude": "290.68\u00b0",
        "NakLord": "Moon",
        "Nakshatra": "Shravana",
        "Sign": "Capricorn",
        "SignLord": "Saturn",
        "Status": "Enemy",
        "house": 10
      },
      "Pluto": {
        "Avastha": "Bala (Child)",
        "Combust": "No",
        "DMS": "26\u00b0 10' 48\"",
        "Degree in sign": 26.180197483422745,
        "Longitude": "236.18\u00b0",
        "NakLord": "Mercury",
        "Nakshatra": "Jyeshtha",
        "Sign": "Scorpio",
        "SignLord": "Mars",
        "Status": "Friendly",
        "house": 8
      },
      "Rahu": {
        "Avastha": "Yuva (Adult)",
        "Combust": "No",
        "DMS": "13\u00b0 27' 19\"",
        "Degree in sign": 13.455328052325678,
        "Longitude": "13.46\u00b0",
        "NakLord": "Venus",
        "Nakshatra": "Bharani",
        "Sign": "Aries",
        "SignLord": "Mars",
        "Status": "--",
        "house": 1
      },
      "Saturn": {
        "Avastha": "Vriddha (Old)",
        "Combust": "Yes",
        "DMS": "23\u00b0 36' 41\"",
        "Degree in sign": 23.611654686159497,
        "Longitude": "83.61\u00b0",
        "NakLord": "Jupiter",
        "Nakshatra": "Punarvasu",
        "Sign": "Gemini",
        "SignLord": "Mercury",
        "Status": "Friendly",
        "house": 3
      },
      "Sun": {
        "Avastha": "Mrita (Dead)",
        "Combust": "No",
        "DMS": "28\u00b0 2' 37\"",
        "Degree in sign": 28.043845148242724,
        "Longitude": "88.04\u00b0",
        "NakLord": "Jupiter",
        "Nakshatra": "Punarvasu",
        "Sign": "Gemini",
        "SignLord": "Mercury",
        "Status": "Neutral",
        "house": 3
      },
      "Uranus": {
        "Avastha": "Yuva (Adult)",
        "Combust": "No",
        "DMS": "12\u00b0 26' 52\"",
        "Degree in sign": 12.448042012709777,
        "Longitude": "312.45\u00b0",
        "NakLord": "Rahu",
        "Nakshatra": "Shatabhisha",
        "Sign": "Aquarius",
        "SignLord": "Saturn",
        "Status": "Friendly",
        "house": 11
      },
      "Venus": {
        "Avastha": "Kumara (Teen)",
        "Combust": "No",
        "DMS": "19\u00b0 10' 17\"",
        "Degree in sign": 19.1714409150115,
        "Longitude": "49.17\u00b0",
        "NakLord": "Moon",
        "Nakshatra": "Rohini",
        "Sign": "Taurus",
        "SignLord": "Venus",
        "Status": "Own Sign",
        "house": 2
      }
    }

navamsa_results = {}

for planet, data in planets.items():
    sign = data["Sign"]
    deg = data["Degree in sign"]
    nav_sign, pada = get_navamsa_sign(sign, deg)
    navamsa_results[planet] = {
        "rashi": sign,
        "degree_in_sign": round(deg, 2),
        "navamsa_pada": pada,
        "navamsa_sign": nav_sign
    }

import json
print(json.dumps(navamsa_results, indent=2))
